In [ ]:
import os
import json
import requests
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt

from functions import load_api_key

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [ ]:
api_key = load_api_key()

# MRT coordinates

In [ ]:
line_ls = ['EW', 'NS', 'NE', 'CC', 'DT', 'TE', 'BP', 'SE', 'SW', 'PE', 'PW']

station_ls = []

for line in line_ls:
    temp_ls = [line + str(i) for i in range(1, 40)]
    station_ls = station_ls + temp_ls
station_ls

In [ ]:
station_df = pd.DataFrame(station_ls, columns = ['station_code'])
station_df['station_name'] = None
station_df['postal_code']  = None
station_df['latitude']     = None
station_df['longitude']    = None
station_df

In [ ]:
start = time.time()
for station in station_ls:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={station}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": api_key}
    response = requests.get(url, headers=headers)
    parsed_data = json.loads(response.text)

    for count in range(len(parsed_data['results'])):  
         if ('MRT' in parsed_data['results'][count]['SEARCHVAL']) or ('LRT' in parsed_data['results'][count]['SEARCHVAL']):
             station_df.loc[station_df['station_code'] == station, 'station_name'] = parsed_data['results'][count]['SEARCHVAL']
             station_df.loc[station_df['station_code'] == station, 'postal_code'] = parsed_data['results'][count]['POSTAL']
             station_df.loc[station_df['station_code'] == station, 'latitude'] = parsed_data['results'][count]['LATITUDE']
             station_df.loc[station_df['station_code'] == station, 'longitude'] = parsed_data['results'][count]['LONGITUDE'] 
    
    print(station, start -  time.time())

In [ ]:
station_df[['station_name', 'postal_code', 'latitude', 'longitude']]

In [ ]:
station_df = station_df.drop_duplicates(subset = ['station_name', 'postal_code', 'latitude', 'longitude'], keep='first')
station_df = station_df.loc[station_df['postal_code'] != 'NIL']
station_df = station_df.dropna()

In [ ]:
station_df